In [ ]:
import numpy as np

In [ ]:
test_text = """##########
#..O..O.O#
#......O.#
#.OO..O.O#
#..O@..O.#
#O#..O...#
#O..O..O.#
#.OO.O.OO#
#....O...#
##########

<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^
vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v
><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<
<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^
^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><
^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^
>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^
<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>
^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>
v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^"""

In [ ]:
with open("input.txt") as f:
    data_text = f.read()

In [ ]:
def get_vector(ch: str) -> tuple:
    if (ch == "^"):
        return (-1, 0)
    if (ch == ">"):
        return (0, 1)
    if (ch == "v"):
        return (1, 0)
    if (ch == "<"):
        return (0, -1)
    
def get_ch(vec: tuple) -> tuple:
    if vec == (-1, 0):
        return "^"
    if vec == (0, 1):
        return ">"
    if vec == (1, 0):
        return "v"
    if vec == (0, -1):
        return "<"

def pase_text(text: str):
    board, instructions = [x.strip() for x in text.split("\n\n")]
    board_array = np.array([[j for j in i] for i in board.split("\n")])
    instr_str_list = [i for i in instructions.replace("\n", "")]
    instr = list(map(get_vector, instr_str_list))
    return board_array, instr

def add_tuple(a, b):
    return(a[0] + b[0], a[1] + b[1])

In [ ]:
class Board:
    def __init__(self, board: np.ndarray):
        self.board = board.copy()
        self.boxes = [(int(x), int(y)) for x, y in zip(*np.where(board == "O"))]
        self.walls = [(int(x), int(y)) for x, y in zip(*np.where(board == "#"))]
        self.pos = tuple(map(int, np.where(board == "@")))
    
    def is_move_valid(self, start_pos: tuple, dp: tuple) -> bool:
        candidate = add_tuple(start_pos, dp)
        if candidate in self.walls:
            return False
        if candidate in self.boxes:
            return self.is_move_valid(candidate, dp)
        return True
        
    def _move_box(self, start_pos: tuple, dp: tuple):
        start_box = add_tuple(start_pos, dp)
        if start_box not in self.boxes:
            return
        candidate = start_box
        while candidate in self.boxes:
            candidate = add_tuple(candidate, dp)
        self.boxes[self.boxes.index(start_box)] = candidate
        return
        
    def move(self, dp: tuple):
        if not self.is_move_valid(self.pos, dp):
            return
        self._move_box(self.pos, dp)
        self.pos = add_tuple(self.pos, dp)
        
    def update(self):
        new_board = np.full_like(self.board, ".")
        for pos in self.walls:
            new_board[pos] = "#"
        for pos in self.boxes:
            new_board[pos] = "O"
        new_board[self.pos] = "@"
        self.board = new_board
    
    def get_score(self) -> int:
        score = 0
        for box_pos in self.boxes:
            score += box_pos[0] * 100 + box_pos[1]
        return score
        

### Part one

In [ ]:
board_arr, instructions = pase_text(data_text)

board = Board(board_arr)
for vec in instructions:
    board.move(vec)
board.get_score()
    

### Part two

In [ ]:
class AmericaBoard(Board):
    def __init__(self, board):
        super().__init__(board)
        self.boxes = [(pos[0], 2* pos[1]) for pos in self.boxes]
        self.walls = [(pos[0], 2* pos[1]) for pos in self.walls]
        self.pos = (self.pos[0], self.pos[1]*2)
        self.board = np.full((self.board.shape[0], self.board.shape[1] * 2), ".")
        self.update()
        
    def update(self):
        new_board = np.full_like(self.board, ".")
        add_pixel = (0, 1)
        for pos in self.walls:
            pos_p = add_tuple(pos, add_pixel)
            new_board[pos] = "#"
            new_board[pos_p] = "#"
        for pos in self.boxes:
            pos_p = add_tuple(pos, add_pixel)
            new_board[pos] = "O"
            new_board[pos_p] = "O"
        new_board[self.pos] = "@"
        self.board = new_board
        
    def is_move_valid_box(self, box_id, dp):
        box_base = self.boxes[box_id]
        box_points = [box_base, add_tuple(box_base, (0, 1))]
        if dp == (0, 1):
            return self.is_move_valid(box_points[1], dp)
        if dp == (0, -1):        
            return self.is_move_valid(box_points[0], dp)
        return self.is_move_valid(box_points[0], dp) and self.is_move_valid(box_points[1], dp)
    
    def is_move_valid(self, start_pos: tuple, dp: tuple) -> bool:
        candidate = add_tuple(start_pos, dp)
        candidate_m = add_tuple(candidate, (0, -1))
        
        if candidate in self.walls or candidate_m in self.walls:
            return False
        if candidate in self.boxes or candidate_m in self.boxes:
            candidate_id = self._get_box_id(candidate)
            return self.is_move_valid_box(candidate_id, dp)
        return True
    
    def _get_box_id(self, candidate):
        candidate_m = add_tuple(candidate, (0, -1))
        for pos in [candidate, candidate_m]:
            if pos in self.boxes:
                return self.boxes.index(pos)
        
    def _check_collisions_box(self, pos: tuple, dp: tuple) -> bool:
        next_pos = add_tuple(pos, dp)
        next_pos_m = add_tuple(next_pos, (0, -1))
        return next_pos not in self.boxes and next_pos_m not in self.boxes
    
    def move(self, dp: tuple):
        if not self.is_move_valid(self.pos, dp):
            return
        next_pos = add_tuple(self.pos, dp)
        if self._check_collisions_box(self.pos, dp):
            self.pos = next_pos
            return
        first_box = self._get_box_id(next_pos)
        self._move_box(first_box, dp)
        self.pos = add_tuple(self.pos, dp)
        
    def _move_box(self, box_id: int, dp: tuple):
        box_base = self.boxes[box_id]
        box_points = [box_base, add_tuple(box_base, (0, 1))]
        box_moved = [add_tuple(box_points[0], dp), add_tuple(box_points[1], dp)]
        if dp == (0, 1):
            next_id = self._get_box_id(box_moved[1])
            if next_id:
                self._move_box(next_id, dp)
            self.boxes[box_id] = add_tuple(self.boxes[box_id], dp)
            return         
        if dp == (0, -1):
            next_id = self._get_box_id(box_moved[0])
            if next_id:
                self._move_box(next_id, dp)
            self.boxes[box_id] = add_tuple(self.boxes[box_id], dp)
            return
        for point in box_moved:
            next_id = self._get_box_id(point)
            if next_id:
                self._move_box(next_id, dp)
        self.boxes[box_id] = add_tuple(self.boxes[box_id], dp)
        
        


In [ ]:
board_arr, instructions = pase_text(data_text)
board = AmericaBoard(board_arr)
bord_america =  board.board
bord_america.shape
for vec in instructions:
    board.move(vec)
board.get_score()